# 내가 구독하는 유튜버 데이터 분석

In [2]:
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

scopes = [ "https://www.googleapis.com/auth/youtube",
"https://www.googleapis.com/auth/youtube.force-ssl",
"https://www.googleapis.com/auth/youtube.readonly",
"https://www.googleapis.com/auth/youtubepartner"]

### Google OAuth 2.0 등록

In [3]:
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "../secret_key/client_secret.json"
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secrets_file, scopes)
credentials = flow.run_console()
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=699806263406-koq1jouhe37k38dvvshpu98ib9m5o53g.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutubepartner&state=YDsj2HHIQwYKKwQcaE8nEiCByq5sPQ&prompt=consent&access_type=offline
Enter the authorization code: 4/yQEaJLzdzxtyRgkEsgh5ezfzUyW8_EcN_5O-oZZjTHDTvGNYR0JxTz0


In [4]:
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

### 구독자 API 요청

In [5]:
my_subscriptions = []

request = youtube.subscriptions().list(
    part='snippet',
    mine='true',
    maxResults='50',
    pageToken=''
)
response = request.execute()

# print(response)

#### 구독자 API 요청 함수

In [6]:
def get_subscriptions():
    my_subscriptions = []
    next_page_token = ''
    
    while(True):
        print('here')
        request = youtube.subscriptions().list(
            part='snippet',
            mine='true',
            maxResults='50',
            pageToken=next_page_token
        )
        response = request.execute()
        my_subscriptions += response['items']
        
        if 'nextPageToken' not in response.keys():
            break
        next_page_token = response['nextPageToken']
    return my_subscriptions

In [7]:
my_subscriptions = get_subscriptions()

here
here
here
here


In [8]:
len(my_subscriptions)

181

In [9]:
my_subscriptions[0]

{'kind': 'youtube#subscription',
 'etag': '"xwzn9fn_LczrfK9QS3iZcGzqRGs/dhNNLrIxUTIz2xok-Ps_LjNHC0k"',
 'id': 'XMm1RFsYKmtAqoa2erMGw1z5bX_IrrDeu4d_EgVAFFc',
 'snippet': {'publishedAt': '2020-03-05T14:41:04.125Z',
  'title': '이서니',
  'description': '',
  'resourceId': {'kind': 'youtube#channel',
   'channelId': 'UCcL8PoeJCC6DwAcKyzw_eQg'},
  'channelId': 'UCWTLxUY-0lkcLHFYz3swO1Q',
  'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/-CpKeQvJySak/AAAAAAAAAAI/AAAAAAAAAAA/rHG34FVuhtU/s88-c-k-no-mo-rj-c0xffffff/photo.jpg'},
   'medium': {'url': 'https://yt3.ggpht.com/-CpKeQvJySak/AAAAAAAAAAI/AAAAAAAAAAA/rHG34FVuhtU/s240-c-k-no-mo-rj-c0xffffff/photo.jpg'},
   'high': {'url': 'https://yt3.ggpht.com/-CpKeQvJySak/AAAAAAAAAAI/AAAAAAAAAAA/rHG34FVuhtU/s800-c-k-no-mo-rj-c0xffffff/photo.jpg'}}}}

## 구독자 정보 Dataframe

In [10]:
import pandas as pd

In [11]:
df_sub = pd.DataFrame(my_subscriptions)

### snippet에 들어있는 tilte, description, channelId, thumbnails을 추가

In [43]:
def extract_snippet_data(df_sub):
    title = []
    description = []
    channelId = []
    thumbnails = []

    for snippet in df_sub['snippet']:
        title.append(snippet['title'])
        description.append(snippet['description'])
        channelId.append(snippet['resourceId']['channelId'])
        thumbnails.append(snippet['thumbnails']['medium']['url'])
        
    df_sub['title'] = title
    df_sub['description'] = description
    df_sub['channelId'] = channelId
    df_sub['thumbnails'] = thumbnails
    
    columns = ['title', 'description', 'channelId',
       'thumbnails', 'etag', 'id', 'kind', 'snippet']
    return df_sub[columns]

In [46]:
df_sub = extract_snippet_data(df_sub)

In [47]:
df_sub.head()

,title,description,channelId,thumbnails,etag,id,kind,snippet
0,이서니,,UCcL8PoeJCC6DwAcKyzw_eQg,https://yt3.ggpht.com/-CpKeQvJySak/AAAAAAAAAAI...,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/dhNNLrIxUTIz2xok-...",XMm1RFsYKmtAqoa2erMGw1z5bX_IrrDeu4d_EgVAFFc,youtube#subscription,"{'publishedAt': '2020-03-05T14:41:04.125Z', 't..."
1,엔마유튜브,제라스장인/메타를 소개하는 남자\n메소남 엔마입니다.\n롤말고 다른것도 많이합니다 ...,UCF-rFAdN7dzd-SI2T3rkCaQ,https://yt3.ggpht.com/-HQdcXajwans/AAAAAAAAAAI...,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/BIvGGGHh-T66W_vMC...",XMm1RFsYKmtAqoa2erMGw2-_zhRy5SidEeP32uxncVg,youtube#subscription,"{'publishedAt': '2020-01-15T09:40:40.624Z', 't..."
2,포프TV,"탑 프로그래머/엔지니어가 즐겨보는 유튜브 채널 포프TV\n\n전 피파, 헤일로, 마...",UC63J0Q5huHSlbNT3KxvAaHQ,https://yt3.ggpht.com/-nGQgbxEwwUg/AAAAAAAAAAI...,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/_cJc-rq-rBUd_jIw8...",XMm1RFsYKmtAqoa2erMGwxdquHhweIV3Xv-t3zVtNLw,youtube#subscription,"{'publishedAt': '2019-01-01T15:53:21.972Z', 't..."
3,승우아빠 일상채널,"🎁 승소리 포장 출력소 🎁\n🗯️ 게임, 일상, 흑우리뷰 🗯️",UCs1R9zP6rH2GlTNHUI5SBkw,https://yt3.ggpht.com/-6XLk13REPnc/AAAAAAAAAAI...,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/rdPrNR38Dp1aQH96R...",XMm1RFsYKmvBkbp-hf4Ryg0vUu7KxIYHgJDa45uTiuA,youtube#subscription,"{'publishedAt': '2020-04-04T13:33:37.056Z', 't..."
4,노마드 코더 Nomad Coders,"한국인 린과 콜롬비아인 니꼴라스의 프로젝트 ""노마드 코더"" 입니다.\n2015년 떠...",UCUpJs89fSBXNolQGOYKn0YQ,https://yt3.ggpht.com/-CTwXMuZRaWw/AAAAAAAAAAI...,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/ff50cG1yssS-McjuJ...",XMm1RFsYKmsKG2GYc6kuBpZuKjG2xWAgDZWcWW9bkSA,youtube#subscription,"{'publishedAt': '2018-11-04T15:02:36.322Z', 't..."


## 구독자 정보 csv로 저장

In [40]:
df_sub.to_csv('my_subscriptions.csv')

In [48]:
read_sub =pd.read_csv('my_subscriptions.csv', index_col = 0)

In [67]:
request = youtube.channels().list(
    part='statistics,snippet,contentDetails,topicDetails,brandingSettings,invideoPromotion',
    id='UCs1R9zP6rH2GlTNHUI5SBkw,UCUpJs89fSBXNolQGOYKn0YQ'
)
response = request.execute()

## 채널 정보 가져오기

In [122]:
import time


In [125]:
def get_channels(df):
    responses= []
    channelIds = ''
    for index, channelId in enumerate(df['channelId']):
        channelIds += "," + channelId
#       10개씩 묶어서 요청        
        if index%10 == 0:
            request = youtube.channels().list(
                part='statistics,snippet,contentDetails,topicDetails,brandingSettings,invideoPromotion',
                id=channelIds)
            channelIds = ""

            response = request.execute()
            responses += response['items']
            time.sleep(3)
            
    if channelIds:
        request = youtube.channels().list(
            part='statistics,snippet,contentDetails,topicDetails,brandingSettings,invideoPromotion',
            id=channelIds)
        response = request.execute()
        responses += response['items']
    return responses

In [126]:
responses = get_channels(df_sub)

## 채널에서 keywords, topicDetails 등 가져오기

In [169]:
def extract_channel_data(responses):
    id_list = []
    keywords = []
    topicIds = []
    topicCategories = []
    videoCount = []
    viewCount = []
    subscriberCount = []

    for index, response in enumerate(responses):
        id_list.append(response['id'])
        if ('keywords' in response['brandingSettings']['channel']):
            keywords.append(response['brandingSettings']['channel']['keywords'])
        else:
            keywords.append('NULL')
            
        if ('topicDetails' in response):
            topicIds.append(response['topicDetails']['topicIds'])
            topicCategories.append(response['topicDetails']['topicCategories'])
        else:
            topicIds.append('NULL')
            topicCategories.append('NULL')
        videoCount.append(response['statistics']['videoCount'])
        viewCount.append(response['statistics']['viewCount'])
        
        if (response['statistics']['hiddenSubscriberCount']):
            subscriberCount.append('0')
        else:
            subscriberCount.append(response['statistics']['subscriberCount'])
   
    df_dict = {
        'channelId': id_list,
        'keywords': keywords,
        'topicIds': topicIds,
        'topicCategories': topicCategories,
        'videoCount': videoCount,
        'viewCount': viewCount,
        'subscriberCount': subscriberCount
    }
    return pd.DataFrame(df_dict)

In [170]:
df_channels = extract_channel_data(responses)

In [171]:
df_channels.to_csv('channels_data.csv')

,channelId,keywords,topicIds,topicCategories,videoCount,viewCount,subscriberCount
0,UCcL8PoeJCC6DwAcKyzw_eQg,"리그오브레전드 이서니 뉴메타 개짓거리 ""롤 유튜버"" 롤토체스 ""롤 방송"" ""롤 공략...","[/m/025zzc, /m/0403l3g, /m/0bzvm2, /m/0403l3g,...",[https://en.wikipedia.org/wiki/Video_game_cult...,3396,145859957,183000
1,UCZVD--cl8FLRn7kmSudAuBA,요리노래 요리랩 노래요리 랩요리 노래레시피 레시피노래 비빔국수 라면볶이 라볶이 돼지...,"[/m/02wbm, /m/019_rr, /m/02wbm]","[https://en.wikipedia.org/wiki/Food, https://e...",15,14792872,335000
2,UCI6bxkc4WHwIHLis1Hg2jTw,"디스커버리 ""디스커버리 채널"" ""디스커버리 채널 코리아"" ""Discovery Kor...","[/m/02jjt, /m/02vxn, /m/0f2f9]","[https://en.wikipedia.org/wiki/Entertainment, ...",133,163846510,963000
3,UCaJdckl6MBdDPDf75Ec_bJA,재테크 부동산 창업 주식 쇼핑몰 스마트스토어 돈버는법 돈버는방법 신사임당 경제,NULL,NULL,397,57747981,683000
4,UCCs1YZHxUTIbLZd-xrvvMCw,뉴메타 청원이 게임 리뷰 실황 순위 랭킹,"[/m/0403l3g, /m/025zzc, /m/0bzvm2, /m/025zzc, ...","[https://en.wikipedia.org/wiki/Action_game, ht...",128,76593818,256000
5,UCYO_jab_esuFRV4b17AJtAw,Mathematics,[/m/01k8wb],[https://en.wikipedia.org/wiki/Knowledge],94,128797177,2700000
6,UCF-rFAdN7dzd-SI2T3rkCaQ,"롤 제라스 ""제라스 장인"" ""롤 제라스 장인"" ""종합 게임 채널"" 메소남 ""메타를소...","[/m/0403l3g, /m/025zzc, /m/0bzvm2, /m/0403l3g,...","[https://en.wikipedia.org/wiki/Action_game, ht...",428,67451737,183000
7,UCpSFkW-1TFYy--Ag7__0w5A,"리그오브레전드 롤 르블랑 블개 ""League Of Legend"" LOL","[/m/025zzc, /m/0403l3g, /m/0bzvm2, /m/0403l3g,...","[https://en.wikipedia.org/wiki/Action_game, ht...",471,86280735,241000
8,UCs1R9zP6rH2GlTNHUI5SBkw,승우아빠,"[/m/025zzc, /m/0403l3g, /m/0bzvm2, /m/025zzc, ...","[https://en.wikipedia.org/wiki/Action_game, ht...",68,3998936,51700
9,UCUpJs89fSBXNolQGOYKn0YQ,코딩 프로그래밍 programming coding 프로그래머 웹개발 개발자 deve...,NULL,NULL,520,5906184,136000


In [173]:
df_sub_channel = pd.merge(df_sub, df_channels, how='outer')

In [174]:
df_sub_channel.to_csv('subscribe_channels_data.csv')